In [1]:
import os
import pandas as pd


In [2]:
# sheet link: https://docs.google.com/spreadsheets/d/1BE3rk5TWH3HWdU9eoh1QZxCdDXN2toHjJ6djDYfWFcQ/edit#gid=130134188
DATA_PATH = os.path.join('mapped_af_NLP.csv')
af_data = pd.read_csv(DATA_PATH)

In [3]:
reversible_tags = af_data
reversible_tags['NLP_framework'] = reversible_tags.apply(
    lambda x: f"{x['Virtual Type']}->{x['Virtual Pillar']}->{x['Virtual Sub-pillar']}", axis=1
)
reversible_tags = reversible_tags[['Framework Title', 'Type', 'Pillar', 'Sub-pillar', 'NLP_framework', 'Reversible']]
reversible_tags

,Framework Title,Type,Pillar,Sub-pillar,NLP_framework,Reversible
0,2020 Okular Analytics Framework,1D,Context,Environment,1D->Context->Environment,YES
1,2020 Okular Analytics Framework,1D,Context,Socio Cultural,1D->Context->Socio Cultural,YES
2,2020 Okular Analytics Framework,1D,Context,Economy,1D->Context->Economy,YES
3,2020 Okular Analytics Framework,1D,Context,Demography,1D->Context->Demography,YES
4,2020 Okular Analytics Framework,1D,Context,Legal/Policy,1D->Context->Legal & Policy,YES
...,...,...,...,...,...,...
472,GIMAC Generic,2D,Stated Priorities and Recommendations,Recommendations from aid/research actors,2D->Priority Interventions->Expressed by human...,YES
473,GIMAC Generic,HIGH LEVEL TAGS,General,Information gap,1D->Information and Communication->Knowledge a...,YES
474,GIMAC Generic,HIGH LEVEL TAGS,General,Response gap,2D->Capacities & Response->Number of people re...,YES
475,GIMAC Generic,HIGH LEVEL TAGS,General,Risk,1D->Shock/Event->Hazard & Threats,YES


In [4]:
reversible_tags['original_tags'] = reversible_tags.apply(
    lambda x: f"{x['Pillar']}->{x['Sub-pillar']}", axis=1
)

<ipython-input-4-28a62280a377>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reversible_tags['original_tags'] = reversible_tags.apply(


In [5]:
grouped_original_tags = reversible_tags.groupby('NLP_framework', as_index=False).agg({'original_tags':lambda x: list(x)})
grouped_original_tags.head()

,NLP_framework,original_tags
0,1D->COVID-19->Cases,"[COVID-19 Overview->Cases, COVID-19 Overview->..."
1,1D->COVID-19->Contact tracing,[COVID-19 Overview->Contact Tracing]
2,1D->COVID-19->Deaths,[COVID-19 Overview->Deaths]
3,1D->COVID-19->Hospitalization & Care,"[COVID-19 Overview->Hospitalization & Care, CO..."
4,1D->COVID-19->Restriction measures,"[COVID-19 Overview->Containment measures, COVI..."


In [11]:
subpillars_1d_af = grouped_original_tags[
    grouped_original_tags.NLP_framework.apply(
        lambda x: x.split('->')[0] == '1D'
    )
]

subpillars_2d_af = grouped_original_tags[
    grouped_original_tags.NLP_framework.apply(
        lambda x: x.split('->')[0] == '2D'
    )
]

In [12]:
def get_matching_dict(df):
    af_to_nlp_dict = {}
    for id, row in df.iterrows():
        for tag_tmp in row['original_tags']:
            af_to_nlp_dict[tag_tmp.lower()] = row['NLP_framework']
    return af_to_nlp_dict

final_matching_dict = {
    '1d_subpillars': get_matching_dict(subpillars_1d_af),
    '2d_subpillars': get_matching_dict(subpillars_2d_af)
}


In [14]:
import json
with open('af_nlp_matching.json', 'w') as fp:
    json.dump(final_matching_dict, fp,  indent=4)

